<div style="text-align: center; background: linear-gradient(135deg, #1a2980, #26d0ce); 
            padding: 30px 25px; border-radius: 10px; color: white; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1); margin-bottom: 20px;">
    <div style="max-width: 600px; margin: 0 auto;">
        <div style="margin-bottom: 15px;">
            <h1 style="color: white; font-size: 2.5em; margin: 0; 
                       font-family: 'Segoe UI', Arial, sans-serif; 
                       text-shadow: 1px 1px 3px rgba(0,0,0,0.2);">
                Stock Price Prediction Report
            </h1>
        </div>
        <div style="margin-bottom: 20px; padding: 0 10px;">
            <p style="font-size: 1.2em; opacity: 0.95; margin: 0; 
                      line-height: 1.5; display: inline;">
                Using Technical Analysis & Machine Learning
            </p>
        </div>
        <div>
            <div style="background-color: rgba(255,255,255,0.15); 
                        padding: 10px 25px; border-radius: 20px; 
                        display: inline-block;">
                <h3 style="color: white; margin: 0; font-weight: 400; 
                           font-size: 1.1em;">
                    <span style="font-weight: 600;">Author:</span> Hoang Son Lai | 
                    <span style="font-weight: 600;">Date:</span> 26/01/2026
                </h3>
            </div>
        </div>
    </div>
</div>

In [26]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import table
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook_connected"
pio.templates.default = "plotly_white"
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from IPython.display import display, clear_output
# python -m nbconvert backend/reports/ml_report.ipynb --to html --no-input
# To convert to html

# Load Data
try:
    df = pd.read_csv('../../data/cleaned/stock_prices.csv')
    df['date'] = pd.to_datetime(df['date'])
    # Sort by Ticker and Date for accurate calculation
    df = df.sort_values(['ticker', 'date'])
except FileNotFoundError:
    print("Error: File 'stock_prices.csv' not found.")

## Part 1. Technical Analysis

This section provides a comprehensive technical analysis of stock prices using multiple indicators. For each stock, I calculate key metrics: 

1. Trend Indicators:

- MA50/MA200: 50-day and 200-day Simple Moving Averages compare current price to medium/long-term trends (Price > MA = bullish +1, Price < MA = bearish -1)

- EMA20: 20-day Exponential Moving Average gives more weight to recent prices for short-term trend direction

2. Momentum Indicators:

- RSI (14-day): Measures overbought (>70 = -1) vs. oversold (<30 = +1) conditions, with 30-70 being neutral

- MACD: Signal line crossover indicator (MACD > 0 = +1 bullish momentum, MACD < 0 = -1 bearish momentum)

3. Volatility & Volume Indicators:

- Bollinger Bands: Price above upper band = overbought (-1), below lower band = oversold (+1), within bands = neutral (0)

- MFI (Money Flow Index): Volume-weighted RSI (>80 = -1 overbought, <20 = +1 oversold, 20-80 = neutral)

Each indicator is scored (+1 for bullish, -1 for bearish, 0 for neutral), with a final aggregate signal determining the overall market outlook (Positive, Negative, or Neutral).

In [30]:
# 1. Define a function to calculate indicators for a single dataframe
def calculate_indicators(data):
    # Ensure data is sorted by date
    data = data.sort_values('date').reset_index(drop=True)
    
    # Trend Indicators
    data.ta.sma(length=50, append=True)  # Simple Moving Average (50)
    data.ta.sma(length=200, append=True) # Simple Moving Average (200)
    data.ta.ema(length=20, append=True)  # Exponential Moving Average (20)
    
    # Momentum Indicators
    data.ta.rsi(length=14, append=True)  # Relative Strength Index (14)
    
    # Volatility Indicators - Bollinger Bands
    # Using bbands with standard parameters
    bbands = data.ta.bbands(length=20, std=2, append=True)
    
    # Trend Following (MACD)
    data.ta.macd(fast=12, slow=26, signal=9, append=True)
    
    # Money Flow Index (MFI)
    data.ta.mfi(length=14, append=True)
    
    # Volume
    data['Volume'] = data['volume']
    
    return data

# 2. Apply calculation to all tickers
# Create a list to store processed dataframes
processed_frames = []

# Get list of unique tickers
tickers = df['ticker'].unique()

for ticker in tickers:
    # Filter data for specific ticker
    ticker_df = df[df['ticker'] == ticker].copy()
    
    # Calculate indicators (need at least 200 days for MA200)
    if len(ticker_df) > 200:
        try:
            ticker_df = calculate_indicators(ticker_df)
            processed_frames.append(ticker_df)
            print(f"✓ Processed {ticker}: {len(ticker_df)} rows, columns: {len(ticker_df.columns)}")
        except Exception as e:
            print(f"✗ Error processing {ticker}: {e}")
    else:
        print(f"✗ Skipped {ticker}: Not enough data ({len(ticker_df)} rows, need at least 200)")

# Combine back into a single main DataFrame
if processed_frames:
    df_ta = pd.concat(processed_frames, ignore_index=True)
    print(f"\nCombined DataFrame shape: {df_ta.shape}")
    print(f"Available columns: {df_ta.columns.tolist()}")
else:
    print("No data was processed!")
    df_ta = pd.DataFrame()

# 3. Check column names and find Bollinger Bands columns
if not df_ta.empty:
    print("\nSearching for Bollinger Bands columns...")
    bollinger_cols = [col for col in df_ta.columns if 'BB' in col or 'bb' in col]
    print(f"Bollinger Bands columns found: {bollinger_cols}")
    
    # Find the actual column names for BB upper and lower
    bb_upper = None
    bb_lower = None
    bb_middle = None
    
    for col in bollinger_cols:
        col_lower = col.lower()
        if 'bbu' in col_lower or 'upper' in col_lower:
            bb_upper = col
        elif 'bbl' in col_lower or 'lower' in col_lower:
            bb_lower = col
        elif 'bbm' in col_lower or 'middle' in col_lower:
            bb_middle = col
    
    print(f"Upper BB column: {bb_upper}")
    print(f"Lower BB column: {bb_lower}")
    print(f"Middle BB column: {bb_middle}")

# 4. Calculate scores for each indicator with safe column access
def calculate_scores(row):
    scores = {}
    
    # Helper function to safely get values
    def get_value(col_name, default=np.nan):
        if col_name in row and pd.notna(row[col_name]):
            return row[col_name]
        return default
    
    # MA50 vs Close
    ma50 = get_value('SMA_50')
    if pd.notna(ma50):
        if row['close'] > ma50:
            scores['MA50'] = 1
        elif row['close'] < ma50:
            scores['MA50'] = -1
        else:
            scores['MA50'] = 0
    
    # MA200 vs Close
    ma200 = get_value('SMA_200')
    if pd.notna(ma200):
        if row['close'] > ma200:
            scores['MA200'] = 1
        elif row['close'] < ma200:
            scores['MA200'] = -1
        else:
            scores['MA200'] = 0
    
    # EMA20 vs Close
    ema20 = get_value('EMA_20')
    if pd.notna(ema20):
        if row['close'] > ema20:
            scores['EMA'] = 1
        elif row['close'] < ema20:
            scores['EMA'] = -1
        else:
            scores['EMA'] = 0
    
    # MACD
    macd = get_value('MACD_12_26_9') or get_value('MACD')
    if pd.notna(macd):
        if macd > 0:
            scores['MACD'] = 1
        elif macd < 0:
            scores['MACD'] = -1
        else:
            scores['MACD'] = 0
    
    # RSI
    rsi = get_value('RSI_14')
    if pd.notna(rsi):
        if rsi > 70:
            scores['RSI'] = -1  # Overbought
        elif rsi < 30:
            scores['RSI'] = 1   # Oversold
        else:
            scores['RSI'] = 0   # Neutral
    
    # Bollinger Bands
    if bb_upper and bb_lower:
        bb_upper_val = get_value(bb_upper)
        bb_lower_val = get_value(bb_lower)
        
        if pd.notna(bb_upper_val) and pd.notna(bb_lower_val):
            if row['close'] > bb_upper_val:
                scores['BB'] = -1   # Overbought (above upper band)
            elif row['close'] < bb_lower_val:
                scores['BB'] = 1    # Oversold (below lower band)
            else:
                scores['BB'] = 0    # Within bands
    
    # MFI (Money Flow Index)
    mfi = get_value('MFI_14')
    if pd.notna(mfi):
        if mfi > 80:
            scores['MFI'] = -1  # Overbought
        elif mfi < 20:
            scores['MFI'] = 1   # Oversold
        else:
            scores['MFI'] = 0   # Neutral
    
    # Calculate total score
    total_score = sum(scores.values()) if scores else 0
    
    # Determine signal
    if total_score > 0:
        signal = 'Positive'
    elif total_score < 0:
        signal = 'Negative'
    else:
        signal = 'Neutral'
    
    # Add scores and signal to row
    row['MA50_Score'] = scores.get('MA50', 0)
    row['MA200_Score'] = scores.get('MA200', 0)
    row['EMA_Score'] = scores.get('EMA', 0)
    row['MACD_Score'] = scores.get('MACD', 0)
    row['RSI_Score'] = scores.get('RSI', 0)
    row['BB_Score'] = scores.get('BB', 0)
    row['MFI_Score'] = scores.get('MFI', 0)
    row['Total_Score'] = total_score
    row['Signal'] = signal
    
    return row

# Apply scoring function if we have data
if not df_ta.empty:
    print("\nCalculating scores for each row...")
    df_ta = df_ta.apply(calculate_scores, axis=1)
    
    # 5. Create a summary report table (latest data for each ticker)
    # Get the latest date for each ticker
    latest_dates = df_ta.groupby('ticker')['date'].max()
    
    # Create summary dataframe
    summary_rows = []
    for ticker, latest_date in latest_dates.items():
        ticker_data = df_ta[(df_ta['ticker'] == ticker) & (df_ta['date'] == latest_date)]
        if not ticker_data.empty:
            latest_row = ticker_data.iloc[0]
            
            summary_rows.append({
                'Ticker': ticker,
                'Close Price': latest_row['close'],
                'Volume': latest_row['volume'],
                'MA50': latest_row.get('SMA_50', np.nan),
                'MA200': latest_row.get('SMA_200', np.nan),
                'EMA20': latest_row.get('EMA_20', np.nan),
                'MACD': latest_row.get('MACD_12_26_9', np.nan),
                'RSI': latest_row.get('RSI_14', np.nan),
                'BB Upper': latest_row.get(bb_upper, np.nan) if bb_upper else np.nan,
                'BB Lower': latest_row.get(bb_lower, np.nan) if bb_lower else np.nan,
                'MFI': latest_row.get('MFI_14', np.nan),
                'MA50 Score': latest_row['MA50_Score'],
                'MA200 Score': latest_row['MA200_Score'],
                'EMA Score': latest_row['EMA_Score'],
                'MACD Score': latest_row['MACD_Score'],
                'RSI Score': latest_row['RSI_Score'],
                'BB Score': latest_row['BB_Score'],
                'MFI Score': latest_row['MFI_Score'],
                'Total Score': latest_row['Total_Score'],
                'Signal': latest_row['Signal']
            })
    
    # Create summary dataframe
    summary_df = pd.DataFrame(summary_rows)
    
    # 6. Display the summary table with styling
    def style_scores(val):
        if isinstance(val, (int, float)):
            if val > 0:
                return 'color: green; font-weight: bold;'
            elif val < 0:
                return 'color: red; font-weight: bold;'
            else:
                return 'color: black;'
        return ''
    
    def style_signal(val):
        if val == 'Positive':
            return 'background-color: #d4edda; color: #155724; font-weight: bold;'
        elif val == 'Negative':
            return 'background-color: #f8d7da; color: #721c24; font-weight: bold;'
        else:
            return 'background-color: #fff3cd; color: #856404; font-weight: bold;'
    
    # Apply styling - using map() instead of applymap() for newer pandas versions
    score_columns = ['MA50 Score', 'MA200 Score', 'EMA Score', 'MACD Score', 
                    'RSI Score', 'BB Score', 'MFI Score', 'Total Score']
    
    # Filter columns that actually exist in the dataframe
    existing_score_cols = [col for col in score_columns if col in summary_df.columns]
    
    # Create styler object
    styler = summary_df.style
    
    # Apply formatting
    styler = styler.format({
        'Close Price': '{:.2f}',
        'MA50': '{:.2f}',
        'MA200': '{:.2f}',
        'EMA20': '{:.2f}',
        'MACD': '{:.4f}',
        'RSI': '{:.1f}',
        'BB Upper': '{:.2f}',
        'BB Lower': '{:.2f}',
        'MFI': '{:.1f}',
        'Volume': '{:,.0f}'
    })
    
    # Apply styling to score columns (use map for newer pandas)
    if existing_score_cols:
        styler = styler.map(style_scores, subset=existing_score_cols)
    
    # Apply styling to signal column
    if 'Signal' in summary_df.columns:
        styler = styler.map(style_signal, subset=['Signal'])
    
    # Set properties and table styles
    styler = styler.set_properties(**{
        'text-align': 'center',
        'border': '1px solid #ddd',
        'padding': '5px'
    })
    
    styler = styler.set_table_styles([
        {'selector': 'thead th', 'props': [
            ('background-color', '#4a6fa5'),
            ('color', 'white'),
            ('font-weight', 'bold'),
            ('padding', '10px'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'tbody td', 'props': [
            ('padding', '8px'),
            ('border', '1px solid #ddd')
        ]},
        {'selector': 'tr:hover', 'props': [('background-color', '#f5f5f5')]}
    ])
    
    styler = styler.set_caption("Technical Analysis Summary - Latest Signals")
    
    # Display the styled table
    print(f"\nSummary table shape: {summary_df.shape}")
    print(f"Columns in summary table: {summary_df.columns.tolist()}")
    display(styler)



✓ Processed AAPL: 1256 rows, columns: 31
✓ Processed ADBE: 1256 rows, columns: 31
✓ Processed AMZN: 1256 rows, columns: 31
✓ Processed BAC: 1256 rows, columns: 31
✓ Processed DIS: 1256 rows, columns: 31
✓ Processed GOOGL: 1256 rows, columns: 31
✓ Processed HD: 1256 rows, columns: 31
✓ Processed JNJ: 1256 rows, columns: 31
✓ Processed JPM: 1256 rows, columns: 31
✓ Processed KO: 1256 rows, columns: 31
✓ Processed MA: 1256 rows, columns: 31
✓ Processed META: 1256 rows, columns: 31
✓ Processed MSFT: 1256 rows, columns: 31
✓ Processed NFLX: 1256 rows, columns: 31
✓ Processed NVDA: 1256 rows, columns: 31
✓ Processed PG: 1256 rows, columns: 31
✓ Processed PYPL: 1256 rows, columns: 31
✓ Processed TSLA: 1256 rows, columns: 31
✓ Processed UNH: 1256 rows, columns: 31
✓ Processed V: 1256 rows, columns: 31
✓ Processed WMT: 1256 rows, columns: 31

Combined DataFrame shape: (26376, 31)
Available columns: ['ticker', 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'SMA_50', 'SMA_200', 'E

,Ticker,Close Price,Volume,MA50,MA200,EMA20,MACD,RSI,BB Upper,BB Lower,MFI,MA50 Score,MA200 Score,EMA Score,MACD Score,RSI Score,BB Score,MFI Score,Total Score,Signal
0,AAPL,248.68,"20,369,381",269.78,234.74,259.27,-6.1921,23.5,280.05,243.12,25.3,-1,1,-1,-1,1,0,0,-1,Negative
1,ADBE,302.73,"1,708,290",331.50,357.65,318.17,-12.2318,34.1,370.32,280.05,32.9,-1,-1,-1,-1,0,0,0,-4,Negative
2,AMZN,239.68,"17,267,451",232.37,220.30,236.04,1.4068,55.5,248.81,224.73,61.6,1,1,1,1,0,0,0,4,Positive
3,BAC,51.58,"14,331,746",53.90,48.30,53.79,-0.6639,34.6,58.25,50.93,45.3,-1,1,-1,-1,0,0,0,-2,Negative
4,DIS,111.56,"4,051,824",109.76,110.60,112.44,0.4457,48.3,115.97,110.68,59.2,1,1,-1,1,0,0,0,2,Positive
5,GOOGL,328.39,"12,237,364",312.72,229.67,323.91,5.7880,58.3,340.02,306.56,58.9,1,1,1,1,0,0,0,4,Positive
6,HD,383.07,"996,841",356.24,370.44,368.76,7.9576,65.6,395.88,331.25,75.4,1,1,1,1,0,0,0,4,Positive
7,JNJ,218.29,"2,194,145",206.78,176.21,212.80,3.7928,67.7,222.94,199.47,60.9,1,1,1,1,0,0,0,4,Positive
8,JPM,297.76,"4,505,715",312.95,288.98,313.03,-3.7925,34.8,341.22,295.53,45.1,-1,1,-1,-1,0,0,0,-2,Negative
9,KO,72.26,"5,750,665",70.56,69.11,70.70,0.4989,62.9,72.96,67.49,62.4,1,1,1,1,0,0,0,4,Positive
